In [63]:
import pandas as pd
x1 = pd.ExcelFile('celebrity_deaths_2016.xlsx')
print(x1.sheet_names)

['celeb_death', 'cause_of_death']


In [64]:
df = x1.parse('celeb_death')
top5 = df.head(5)
print(top5)

  date of death             name age  \
0    2016-01-01        Tony Lane  71   
1    2016-01-01   Gilbert Kaplan  74   
2    2016-01-01      Brian Johns  79   
3    2016-01-01  Natasha Aguilar  45   
4    2016-01-01     Fazu Aliyeva  83   

                                                 bio  cause_id  
0              American art director (Rolling Stone)    8915.0  
1                 American conductor and businessman    2860.0  
2  Australian company director, managing director...    2860.0  
3  Costa Rican swimmer, silver and bronze medalis...   33897.0  
4                   Russian Avar poet and journalist   10648.0  


c:\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [66]:
df_types = df.dtypes
df_shape = df.shape
print(df_types)
print(df_shape)

date of death    datetime64[ns]
name                     object
age                      object
bio                      object
cause_id                float64
dtype: object
(6594, 5)


In [71]:
df = df.drop_duplicates()

In [72]:
cause_of_death = x1.parse('cause_of_death')
cause_top_5 = cause_of_death.head(5)
print(cause_top_5)

   cause_id cause of death
0       753            ALS
1      1039           bomb
2      1120           shot
3      1499           fall
4      1629           shot


c:\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [77]:
cause_of_death.drop_duplicates(subset='cause_id')

,cause_id,cause of death
0,753,ALS
1,1039,bomb
2,1120,shot
3,1499,fall
4,1629,shot
...,...,...
344,116752,injuries sustained in Brussels Airport bombings
345,119640,complications from a fall and Alzheimer's dis...
346,129613,complications from esophageal cancer and pneu...
347,133339,pneumonia as a complication from a kidney inf...


In [78]:
df = df.merge(cause_of_death, how='left', on='cause_id')
df_top5 = df.head(5)
print(df_top5)

  date of death             name age  \
0    2016-01-01        Tony Lane  71   
1    2016-01-01   Gilbert Kaplan  74   
2    2016-01-01      Brian Johns  79   
3    2016-01-01  Natasha Aguilar  45   
4    2016-01-01     Fazu Aliyeva  83   

                                                 bio  cause_id  \
0              American art director (Rolling Stone)    8915.0   
1                 American conductor and businessman    2860.0   
2  Australian company director, managing director...    2860.0   
3  Costa Rican swimmer, silver and bronze medalis...   33897.0   
4                   Russian Avar poet and journalist   10648.0   

               cause of death  
0                brain cancer  
1                      cancer  
2                      cancer  
3   complications of a stroke  
4               heart failure  


In [84]:
df['age'] = pd.to_numeric(df['age'], errors='coerce').dropna()

In [85]:
avg_age = df['age'].mean()
print(avg_age)

76.99418159546777


In [86]:
count = len(df[df['age'] > 70].index)
print(count)

4760


In [89]:
df_youngest = df[df['age'] == df['age'].min()]
youngest_name = df_youngest.iloc[0]['name']
youngest_cause = df_youngest.iloc[0]['cause of death']
print(youngest_name)
print(youngest_cause)

Vautour
 euthanised


In [94]:
df['bio'] = df['bio'].astype(str)

In [95]:
df_american = df[df['bio'].str.contains('American')]
count_american = len(df_american.index)
print(count_american)

2143


In [99]:
import random
random.seed(0)

df_died_50 = df[
    (df['age'] == 50) &
    (not df['bio'].isnull().values.any()) &
    (not df['bio'].str.contains('unknown').any())]